<a href="https://colab.research.google.com/github/hukim1112/MLDL/blob/master/object_detection/generate_anchors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import itertools
import math
import tensorflow as tf

def generate_default_boxes(param):
    """ Generate default boxes for all feature maps

    Args:
        config: information of feature maps
            scales: boxes' size relative to image's size
            fm_sizes: sizes of feature maps
            ratios: box ratios used in each feature maps

    Returns:
        default_boxes: tensor of shape (num_default, 4)
                       with format (cx, cy, w, h)
    """

    default_boxes = []
    scales = param['scales']
    fm_sizes = param['fm_sizes']
    ratios = param['ratios']

    for m, fm_size in enumerate(fm_sizes):
        for i, j in itertools.product(range(fm_size), repeat=2):
            cx = (j + 0.5) / fm_size
            cy = (i + 0.5) / fm_size
            default_boxes.append([
                cx,
                cy,
                scales[m],
                scales[m]
            ])

            default_boxes.append([
                cx,
                cy,
                math.sqrt(scales[m] * scales[m + 1]),
                math.sqrt(scales[m] * scales[m + 1])
            ])

            for ratio in ratios[m]:
                r = math.sqrt(ratio)
                default_boxes.append([
                    cx,
                    cy,
                    scales[m] * r,
                    scales[m] / r
                ])

                default_boxes.append([
                    cx,
                    cy,
                    scales[m] / r,
                    scales[m] * r
                ])

    default_boxes = tf.constant(default_boxes)
    default_boxes = tf.clip_by_value(default_boxes, 0.0, 1.0)

    return default_boxes


In [5]:
anchor_param = {"ratios": [[2], [2, 3], [2, 3], [2, 3], [2]],
                           "scales": [0.1, 0.3, 0.5, 0.7, 0.9, 1.075],
                           "fm_sizes": [28, 14, 7, 3, 1],
                           "image_size": 224} #anchor parameters

In [6]:
anchors = generate_default_boxes(anchor_param)

In [8]:
print(anchors.shape)
# the number of reference boxes is 4664.
# 정규화(0~1.0)된 좌표 cx, cy, w, h로 구성되어 있다.

(4664, 4)
